# Instalação

In [ ]:
!pip install xcompact3d_toolbox

# Escoamento ao longo de um corpo complexo

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import xcompact3d_toolbox as x3d

## Parâmetros

+ Precisão numérico

Se Xcompact3d foi compilada com a flag `-DDOUBLE_PREC` deve se usar `np.float64`, para os outros casos deve-se usar `np.float32`.

In [2]:
x3d.param["mytype"] = np.float64

* Xcompact3d's parâmetros

Para mais informações veja [API reference](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.parameters.Parameters).

In [3]:
prm = x3d.Parameters(
    filename="input.i3d",
    itype=12,
    p_row=0,
    p_col=0,
    nx=257,
    ny=129,
    nz=32,
    xlx=15.0,
    yly=10.0,
    zlz=3.0,
    nclx1=2,
    nclxn=2,
    ncly1=1,
    nclyn=1,
    nclz1=0,
    nclzn=0,
    iin=1,
    re=300.0,
    init_noise=0.0125,
    inflow_noise=0.0125,
    dt=0.0025,
    ifirst=1,
    ilast=45000,
    ilesmod=1,
    iibm=2,
    nu0nu=4.0,
    cnu=0.44,
    irestart=0,
    icheckpoint=45000,
    ioutput=200,
    iprocessing=50,
    jles=4,
)

## Configuração

### Geométria

Todo o que se precisa é um dicionário de Arrays (veja [API reference](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.sandbox.init_epsi)):

In [4]:
epsi = x3d.init_epsi(prm)

As quatro matrizes $\epsilon$ são salvas em um dicionário:

In [ ]:
epsi.keys()

Como exemplo, será desenhado e graficado um cilindro. Para fazer um gráfico `xarray.DataArray` é mais simples com `da.plot()`. Por exemplo, para selecionar um valor em $z$ e fazer um gráfico 2D:

In [ ]:
for key in epsi:
    epsi[key] = epsi[key].geo.cylinder(x=1, y=prm.yly / 4.0)

epsi["epsi"].sel(z=0, method="nearest").plot(x="x");

Nota-se que as geométrias são adicionadas por padrão, todavia, ela pode ser revertida definindo `remp=False`. Vários métodos podem ser executados em cadeia, resultando em uma geométria mais complexa.

In [ ]:
for key in epsi:
    epsi[key] = (
        epsi[key]
        .geo.cylinder(x=2, y=prm.yly / 8.0)
        .geo.cylinder(x=2, y=prm.yly / 8.0, radius=0.25, remp=False)
        .geo.sphere(x=6, y=prm.yly / 4, z=prm.zlz / 2.0)
        .geo.box(x=[3, 4], y=[3, 4])
    )

epsi["epsi"].sel(z=prm.zlz / 2, method="nearest").plot(x="x");

Outro exemplo é o corpo de Ahmed:

In [ ]:
for key in epsi:
    epsi[key] = epsi[key].geo.ahmed_body(x=10, wheels=False)

epsi["epsi"].sel(z=prm.zlz / 2, method="nearest").plot(x="x");

Dando um zoom:

In [ ]:
epsi["epsi"].sel(x=slice(8, None), y=slice(None, 4)).sel(z=prm.zlz / 2, method="nearest").plot(x="x");

Como exemplo, pode-se espelha-lo:

In [ ]:
for key in epsi:
    epsi[key] = epsi[key].geo.mirror("y")

epsi["epsi"].sel(z=prm.zlz / 2, method="nearest").plot(x="x");

Para uma descrição completa das geométrias avaliadas veja a [referência da Api](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.sandbox.Geometry). Observe que se pode combinar diferentes rotinas para criar geométrias únicas, ou criar rotinas únicas para seus objetos.

Começando com uma geométria simples:

In [ ]:
epsi = x3d.sandbox.init_epsi(prm)

for key in epsi:
    epsi[key] = epsi[key].geo.cylinder(x=prm.xlx / 3, y=prm.yly / 2)

epsi["epsi"].sel(z=0, method="nearest").plot(x="x")
plt.show();

O próximo passo é criar os arquivos auxiliares descrevendo as geométrias, para que o Xcompact3d possa lê-lo.

In [ ]:
%%time
dataset = x3d.gene_epsi_3d(epsi, prm)

prm.nobjmax = dataset.obj.size

dataset

### Condição de contorno

Tudo que se precisa é de um Dataset (veja a [referência da API](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.sandbox.init_dataset)):

In [13]:
ds = x3d.init_dataset(prm)

Com dados e atributos anexados, pode-se interagir pelos ícones:

In [ ]:
ds

**Perfil de entrada**: Como as condições de contorno para a velocidade no topo e no fundo do domínio são de deslizamento-livre (`ncly1=nclyn=1`), o perfil de entrada é fluxo contínuo.

In [ ]:
fun = xr.ones_like(ds.y)

# This attribute will be shown in the figure
fun.attrs["long_name"] = r"Inflow Profile - f($x_2$)"

fun.plot();

Reiniciando o plano de entrada `ds[key] *= 0.0`, só para guarantir consistência em multiplas execuções dessa célula. Observe que `ds[key] = 0.0` pode-se reescrever todos os metadados contidos no array, por isso devi-se evita-lo. Na célula a baixo se adicionou-se o perfil de entrada e fazer o gráfico dele como referência:

In [ ]:
for key in "bxx1 bxy1 bxz1".split():
    #
    print(ds[key].attrs["name"])
    #
    ds[key] *= 0.0
    #
    if key == "bxx1":
        ds[key] += fun
    #
    ds[key].plot()
    plt.show()

plt.close("all")

Um ruído branco será aplicado np perfil de entrada, pode-se usar uma função de modulação `mod` para controlar onde vai ser aplicado. Neste caso o ruído será concentrado na região central e definir com zero em $y=0$ e $y=L_y$. O domínio é periodico em $z$ (`nclz1=nclzn=0`). A função fica como:

$$
\text{mod} = \exp\left(-0.2 (y - 0.5 L_y)^2 \right).
$$

Veja o código:

In [ ]:
# Ruído branco para uma semente fixado,
# importante para reprodução, desenvolvimento e debugging
if prm.iin == 2:
    np.random.seed(seed=67)

mod = np.exp(-0.2 * (ds.y - ds.y[-1] * 0.5) ** 2.0)

# Esse atributo será mostrado na figura
mod.attrs["long_name"] = "Noise modulation"

mod.plot();

Novamente, reiniciando o array `ds['noise_mod_x1'] *= 0.0`. Observe que `ds['noise_mod_x1'] *= 0.0` pode reescrever todos os metadados contidos dentro do array, por isso deve ser evitado. Então, se adicionou um perfil de modulação para o array e se faz um gráfico como referência:

In [ ]:
ds["noise_mod_x1"] *= 0.0
ds["noise_mod_x1"] += mod
ds.noise_mod_x1.plot();

Observe que uma das vantagens de usar [xarray](http://docs.xarray.dev/en/stable), `mod`, com a componente (`ny`), sendo automaticamente transmitida para todo ponto em `z` em `ds.noise_mod_x1`, com forma (`ny`, `nz`).

### Condição inicial

Agora se reinicia o campo de velocidade `ds[key] *= 0.0`.

Se adiciona um ruído branco com a forma correta, multiplicado pela amplitude do ruído na condição inicial `init_noise` e multiplicar novamente pela função de modulação `mod`, definida previamente. Finalmente, se adiciona o perfil de fluxo contínuo `fun` em `ux` e fazer os gráficos como referência:

In [ ]:
for key in "ux uy uz".split():
    #
    print(ds[key].attrs["name"])
    #
    ds[key] *= 0.0
    ds[key] += prm.init_noise * (np.random.random(ds[key].shape) - 0.5)
    ds[key] *= mod
    #
    if key == "ux":
        ds[key] += fun
    #
    ds[key].sel(z=slice(None, None, ds.z.size // 3)).plot(x="x", y="y", col="z", col_wrap=2)
    plt.show()
    #

plt.close("all")

## Salvando no disco

é simples como:

In [20]:
prm.dataset.write(ds)

In [21]:
prm.write()

## Rodando a simulação

O objetivo desse notebook é mostrar a capacidade do `xcompact3d_toolbox.sandbox`. No entanto, não esqueça a estabilidade numérica do solver numérico do Xcompact3d **é responsabilidade do usuário achar o conjunto certo de parâmetros físico e numérico**.

Verifique se as flags e opções de compilação no `Makefile` estão de acordo com o esperado. Então, copile o código com o comando `make`.

Finalmente, entre com a seguinte linha com o comando:

```bash
mpirun -n [number of cores] ./xcompact3d |tee log.out
```